### Dataset
1. In **Project 1** we work with “20 Newsgroups” dataset. It is a collection of approximately 20,000 newsgroup documents, partitioned (nearly) evenly across 20 different newsgroups, each corresponding to a different topic.
2. To manually load the data, you need to run this python code. <a href="https://www.dropbox.com/s/5oek8qbsge1y64b/fetch_data.py?dl=0">link to fetch_data.py</a>
3. Easiest way to load the data is to use the built-in dataset loader for "20 newsgroups" from scikit-learn package.


In [1]:
from sklearn.datasets import fetch_20newsgroups
categories = ['comp.graphics','comp.os.ms-windows.misc']
twenty_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

In [3]:
# list of category indices of the documents
type(twenty_train.target) # twenty_train['target']

numpy.ndarray

In [8]:
# so index 0 corresponds to 'comp.graphics'; 1 to 'comp.sys.mac.hardware'
print (twenty_train.target_names) # twenty_train['target_names']

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [23]:
print twenty_train.target_names[twenty_train.target[0]]

comp.sys.mac.hardware


The files themselves are loaded in memory in the data attribute.

In [24]:
print len(twenty_train.data)
print len(twenty_train.filenames)
print len(twenty_train.target)
print len(twenty_train.target_names)

1162
1162
1162
2


## Extracting features from text files

### CountVectorizer
Convert a collection of text documents to a matrix of token counts

$\begin{pmatrix}tf(t_1, d_1) & \cdots & tf(t_m, d_1) \\ tf(t_1, d_2) & \cdots & tf(t_m, d_2) \\ \vdots & \vdots & \vdots \\ tf(t_1, d_n) & \cdots & tf(t_m, d_n) \end{pmatrix}$

$tf(t, d)$: term frequency of term $t$ in the document $d$, i.e. the number of occurrances of term $t$ in the document $d$.

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=1)
vectorizer

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

For the detailed documentation of `CountVectorizer`, see

http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

Note: `fit`, `fit_transform`, `transform` are common methods of all kinds of data transformers in `scikit learn`

In [34]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()

# recall that X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print X_train_tfidf.shape
print '-' * 20
print X_train_counts.toarray()[:30,:5]
print '-' * 20
print X_train_tfidf.toarray()[:30,:5]

(1162, 19610)
--------------------
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 1 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
--------------------
[[ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        

## Training a classifier

Let's train a classifier to predict the category of a post.

In [35]:
# We use Naive Bayes classifier as an example
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

In [36]:
docs_new = ['He is an OS developer', 'OpenGL on the GPU is fast']

X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

'He is an OS developer' => comp.sys.mac.hardware
'OpenGL on the GPU is fast' => comp.graphics


## Dimensionality reduction

NMF: Non-negative matrix factorization
$$\min_{W, H}||X - WH||_F^2 \\ s.t. W \ge 0 \\ H \ge 0$$

In [37]:
from sklearn.decomposition import NMF

model = NMF(n_components=50, init='random', random_state=0)
W_train = model.fit_transform(X_train_tfidf)

In [38]:
print W_train.shape

(1162, 50)


In [39]:
H = model.components_
H.shape

(50, 19610)

In [40]:
H

array([[ 0.01737734,  0.        ,  0.        , ...,  0.        ,
         0.01977479,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.01326934,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.01061516,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.02491158,  0.        ,  0.00064097, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.03186904,  0.04901559,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [41]:
W_test = model.transform(X_new_tfidf)

clf = MultinomialNB().fit(W_train, twenty_train.target)

predicted = clf.predict(W_test)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

'He is an OS developer' => comp.sys.mac.hardware
'OpenGL on the GPU is fast' => comp.graphics
